In [ ]:
!pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=402828da30d94f9bb56c32229a367a54c35d0b1ed6b3b03a71a20cc5602b6870
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [ ]:
import re
import pandas as pd
import json
import gensim.parsing.preprocessing as gsp
from collections import Counter
from stop_words import get_stop_words

# Define preprocessing filters
preprocessing_filters = [
    gsp.strip_tags,
    gsp.strip_punctuation,
    gsp.strip_multiple_whitespaces,
    gsp.strip_numeric,
    gsp.strip_non_alphanum
]

# Function to apply preprocessing filters to a text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs using regular expression
    text = re.sub(r'http\S+', '', text)

    # Remove characters immediately after '@'
    text = re.sub(r'@\S+', '', text)

    # Apply other preprocessing filters
    for filter_func in preprocessing_filters:
        text = filter_func(text)

    # Remove Dutch, French, and English stop words
    stop_words_nl = get_stop_words('dutch')
    stop_words_fr = get_stop_words('french')
    stop_words_en = get_stop_words('english')
    stop_words = set(stop_words_nl + stop_words_fr + stop_words_en)
    return ' '.join([word for word in text.split() if word not in stop_words])

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/memgraph-query-results-export.csv')

# Extract the 'full_text' values from the 't' column and concatenate them
concatenated_text = ' '.join(df['t'].apply(lambda x: json.loads(x)['properties']['full_text']).tolist())

# Read the words to remove from the file
with open("/content/stopwords-nl.txt", "r") as file:
    words_to_remove = {line.strip() for line in file}

# Apply preprocessing filters to the concatenated text
preprocessed_text = preprocess_text(concatenated_text)

# Remove specified words and words shorter than 3 characters
words = [word for word in preprocessed_text.split() if word not in words_to_remove and len(word) > 3]

# Count the frequency of each word
word_counter = Counter(words)

# Get the top 20 most common words
top_20_words = word_counter.most_common(20)

# Print the top 20 most common words
for word, frequency in top_20_words:
    print(word, frequency)


vivaldi 220
jaar 192
regering 173
minister 152
gaat 130
vandaag 128
belgië 124
mensen 124
parlement 116
land 113
hamas 107
vlaamse 102
vlaanderen 100
defensie 100
goed 98
croo 87
beste 83
nieuwe 81
komt 81
vlaams 77
